I've never dealt with accelerometer data, so the first step as a literature search. I found
a lot of papers, but the best (most applicable) were

[1] Gait Identification Using Accelerometer on Mobile Phone, Thang et. al.

[2] Unobtrusive User-Authentication on Mobile Phones using Biometric Gait Recognition, Derawi, et. al.

I mainly used the algorithm from [1] for data analysis, and I used [2] for context on the data - I found the writing a little easier to understand. The algorithm was less performant, however, which is why I used [1].

I used

Physical Activity Recognition Dataset Using Smartphone Sensors, Shoaib et. al. (2013)

for my 'other' dataset.

In [1]:
# Import statements and function defs

import numpy as np
import pandas as pd
import scipy.signal as s

def generate_and_preprocess(dataset):
    """
    Generates a dictionary of data from a dataset in a specific way.
    
    :param dataset: String that is the name/path of the dataset we want to extract
    :return: Dictionary of data, t, x, y, z
    """
    data = pd.read_csv(dataset)
    t = np.array(data["time"])

    accel_headers = ['ax', 'ay', 'az']

    x = np.array(data[accel_headers[0]])
    y = np.array(data[accel_headers[1]])
    z = np.array(data[accel_headers[2]])

    total_time = t[-1] - t[0]
    ends = 0.1 * total_time  # discard beginning and end of dataset because of irregularities

    t_start = t[0] + ends
    t_end = t[-1] - ends

    start_idx = np.where(t <= t_start)[0][-1]
    end_idx = np.where(t >= t_end)[0][0]

    t = t[start_idx:end_idx]
    x = x[start_idx:end_idx]
    y = -y[start_idx:end_idx]
    z = -z[start_idx:end_idx]  # In [1] they oriented their phone differently so I'm adjusting my data. Could use
                               # argrelmin instead of argrelmax but this way I can compare directly.
    out_dict = {
        't': t,
        'x': x,
        'y': y,
        'z': z
    }
    return out_dict


def apply_wavelet_decomp(signal):  # following (1) - get rid of HF because they don't use it
    """
    Applies the wavelet decomposition as in [1]. We simply discard the high frequency contribution each time
    since it's not used.
    
    :param signal: The input signal, S(n) in [1]
    :return: Returns the low-pass filtered signal, what [1] calls 'coarse coefs'
    """
    b = s.daub(6)
    LF = s.decimate(s.convolve(signal, b), 2, zero_phase=True)

    return LF


def apply_l_decomp(signal, l):
    """
    We need to apply wavelet decomposition many times, so this allows us to apply it l times.
    
    :param signal: Input signal
    :param l: Number of times (levels/layers) to apply the wavelet decomposition
    :return: Returns the output from applying decomposition l times.
    """
    lf = signal
    for i in range(l):
        lf = apply_wavelet_decomp(lf)

    return lf


def apply_wavelet_reconstruct(signal):
    """
    Again, follows [1] by simply upsampling the signal.
    
    :param signal: 
    :return: 
    """
    resampled = s.resample(signal, len(signal)*2)
    return resampled


def apply_l_recon(signal, l):
    """
    Like applying the decomposition, we want to apply it l times, so this is a convenience function to do this.
    
    :param signal: Input signal
    :param l: Number of times (levels/layers) to apply reconstruction
    :return: Returns the output from applying the reconstruction l times.
    """
    resampled = signal
    for i in range(l):
        resampled = apply_wavelet_reconstruct(resampled)

    return resampled


def filter_true_peaks(signal, k=1./3.):
    """
    Takes all the peaks found in a signal (as in [1]), finds the mean and standard deviation, and calculates a threshold
    based on that which allows us to determine if peaks are so-called 'true peaks'
    
    :param signal: 
    :param k: 
    :return: 
    """
    peaks = s.argrelmax(signal, order=10)[0]
    peak_heights = signal[peaks]

    threshold = np.mean(peak_heights) + k * np.std(peak_heights)
    true_peaks = peak_heights[np.where(peak_heights > threshold)[0]]
    indices = [np.where(signal == tru)[0][0] for tru in true_peaks]

    return indices


def euclidean_distance(x1, x2):
    """
    Standard Euclidean distance between two points
    """
    return np.sqrt(np.abs(x1**2 - x2**2))


def taxicab_distance(x, y):
    """
    Taxicab distance between two points.
    """
    return np.abs(x - y)


def dtw_ij(cycle_i, cycle_j, dist_fn=taxicab_distance):
    """
    Implementation of wikipedia's example of DTW distance. I ended up using a better implementation but I left this
    here for posterity (and in case the pydtw install doesn't work)
    
    :param cycle_i: The first cycle
    :param cycle_j: The second cycle
    :param dist_fn: The function used to compute the distance between cycle points
    :return: Returns the DTW distance between cycles
    """
    n = len(cycle_i)+1
    m = len(cycle_j)+1
    DTW = np.zeros((n, m))

    for i in range(n):
        DTW[i, 0] = 1e20
    for i in range(m):
        DTW[0, i] = 1e20

    DTW[0, 0] = 0.0

    for i in np.arange(1, n):
        for j in np.arange(1, m):
            cost = dist_fn(cycle_i[i-1], cycle_j[j-1])
            DTW[i, j] = cost + np.min((DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1]))

    return DTW[-1, -1]


def dtw(cycles):
    """
    Runs the DTW distance for each cycle in cycles.
    
    :param cycles: An array of cycles over which to apply the dtw function.
    :return: Returns a matrix of distances from dtw_ij
    """
    use_pydtw = True
    try:
        import pydtw
    except ImportError:
        print("This will take a long time - it would be faster to install pydtw and re-run the program.")
        use_pydtw = False

    N = len(cycles)
    DTW = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if use_pydtw:
                cost, aligned_a, aligned_b = pydtw.dtw1d(cycles[i], cycles[j])
                DTW[i, j] = cost[-1, -1]  # the cost matrix's bottom right entry is the distance between the two cycles
            else:
                DTW[i, j] = dtw_ij(cycles[i], cycles[j])

    return DTW


def extract_avg_cycle(true_peak_indices, t, signal):
    """
    Extracts the average cycle from a list of cycles. Uses dtw to calculate the distances and as in [1]
    uses that to find the average gait cycle.
    
    :param true_peak_indices: The peak indices returned from filter_true_peaks as a 1D array
    :param t: The time signal as a 1D array
    :param signal: The accelerometer signal in 1 direction as a 1D array
    :return: The average walk cycle given the input signal, its index, the difference in time steps, a list of all the 
             walk cycles in the data, and the time array that holds the times of each true peak.
    """
    dt = np.diff(t)
    dt_avg = np.average(dt)
    time = [list(t[true_peak_indices[i]:true_peak_indices[i + 1] + 1]) for i in range(len(true_peak_indices) - 1)]
    cycles = [list(signal[true_peak_indices[i]:true_peak_indices[i+1]+1]) for i in range(len(true_peak_indices)-1)]
    cycles_len = [len(cycle) for cycle in cycles]
    mean = np.mean(cycles_len)
    std = np.std(cycles_len)

    for cycle in cycles:  # get rid of outliers
        if len(cycle) > mean + 1.5 * std or len(cycle) < mean - 1.5 * std:
            cycles.remove(cycle)

    for i in range(len(cycles)):  # normalize everything to between +/- 1
        cycles[i] /= np.max(cycles[i])

    dist = dtw(cycles)
    N = len(cycles)

    d = 1./(N - 1.) * np.sum(dist, axis=1) - np.diagonal(dist)

    avg_index = np.where(d == np.min(d))[0][0]
    avg_cycle = cycles[avg_index]

    return avg_cycle, avg_index, dt_avg, cycles, time


def extract_fourier_features(signal, dt, max_f=6):
    """
    Extracts the fourier features of a smoothed signal. Max_f indicates where to cut off the spectrum so that
    we aren't dealing with a bunch of extra points.
    
    :param signal: The signal (hopefully smoothed)
    :param dt: The time step between signal points, used for computing frequency
    :param max_f: The cutoff frequency for the data returned.
    :return: Frequencies up to max_f and corresponding normalized fourier features.
    """
    freqs = np.abs(np.fft.fftfreq(len(signal), dt))
    fourier = np.abs(np.fft.rfft(signal)) / 1e3
    max_index = np.where(freqs >= max_f)[0][0]
    freqs = s.decimate(freqs[:max_index], 3, zero_phase=True)
    fourier = s.filtfilt(s.gaussian(100, 2.), [1.0], fourier)  # smooth fourier signal before subsampling
    fourier = s.decimate(fourier[:max_index], 3, zero_phase=True)
    return freqs, fourier/np.max(fourier)

In [2]:
# Plotting the data, results of the algorithm
from bokeh.plotting import figure, gridplot
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import Arrow, VeeHead

output_notebook()

data_dict = generate_and_preprocess("data/longwalk.csv")
lf = apply_l_decomp(data_dict['z'], 3)
lf = apply_l_recon(lf, 3)

peaks = filter_true_peaks(lf)
avg_cycle, avg_index, dt_avg, cycles, time = extract_avg_cycle(peaks, data_dict['t'], lf)
freqs, fourier = extract_fourier_features(lf, dt_avg, max_f=6)

N = len(cycles)

p = figure(height=500, width=500)
p2 = figure(height=500, width=500)
start = time[avg_index][0]
for i in range(N):
    if i == avg_index:
        continue
    else:
        length = np.min([len(time[avg_index]), len(cycles[i])])
        p.line(time[avg_index][:length]-start, cycles[i][:length], alpha=0.5)
p.line(time[avg_index]-start, avg_cycle, line_width=3, color='orange', alpha=0.9, legend="Average Gait Cycle")
p.xaxis.axis_label = "Time, Normalized to Average Cycle (s)"
p.yaxis.axis_label = "Z acceleration, Normalized (~ m/s^2)"
p.title.text = "Gait Cycles (My dataset)"
p.legend.location = 'bottom_left'


p2.line(freqs, fourier)
p2.xaxis.axis_label = "Frequency (Hz)"
p2.yaxis.axis_label = "Response (arb)"
p2.title.text = "Fourier Features of smoothed Z acceleration (My dataset)"

s1 = gridplot([[p, p2]])
show(s1, notebook_handle=True)

Loading BokehJS ...

You might notice that I smoothed and subsampled the fourier features - this is so that we can use both the average gait and the fourier features in a classification situation. Because we have so much data (i.e. each vector is of length ~100), an SVM won't be as efficient as a CNN. We'll re-interpret the extracted features as simply a vector and run them through a shallow CNN for classification.

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as Funct


def num_flat_features(x):
    """
    Takes a tensor and counts the number of features in every dimension besides the batch
    dimension
    
    :param x: Input tensor (batch x c x h x w) 
    :return: c*h*w
    """
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features


def to_Variable(numpy_array):
    """
    Takes a 1D numpy array and turns it into a 4D tensor (again, for pytorch's call rules)
    
    :param numpy_array: 1D numpy array, (n)
    :return: 4D Variable(torch.FloatTensor), (1 x 1 x 1 x n)
    """
    return Variable(torch.from_numpy(numpy_array).float()).unsqueeze(0).unsqueeze(1).unsqueeze(2)


class GaitID(nn.Module):
    """
    Subclass of nn.Module that defines the net architecture we're using to identify the owner based
    on their gait data. We want to use the gait as well as the fourier features of the gait, so 
    we take the graph in the following way (left to right):
    
    gait    ---(conv,pool,relu) x 2----\
                                        hidden ===== output
    fourier ---(conv,pool,relu) x 2----/
    
    where we take convolutions and pass through a linear layer for gait and fourier and then
    simply add them together and pass through a final output layer.
    """
    def __init__(self,
                 gait_input_size,
                 fourier_input_size,
                 hidden_size=100,
                 output_size=2  # owner or not (pass through softmax)
                 ):
        super(GaitID, self).__init__()

        self.gait_input_size = gait_input_size
        self.fourier_input_size = fourier_input_size
        self.hidden_size = int(hidden_size)
        self.output_size = int(output_size)

        self.gait_to_hidden = nn.Sequential(
            nn.Conv1d(1, 64, 7, padding=3),  # keep size the same through conv layers
            nn.MaxPool1d(2, stride=2),  # downsample by factor of 2
            nn.ReLU(),
            nn.Conv1d(64, 128, 3, padding=1),
            nn.MaxPool1d(2, stride=2),
            nn.ReLU()  # the size is now 128 x input_size/4 x 1
        )

        self.gait_conv_to_hidden = nn.Sequential(
            nn.Linear(128 * int(self.gait_input_size / 4), self.hidden_size),
            nn.ReLU()
        )

        self.fourier_to_hidden = nn.Sequential(
            nn.Conv1d(1, 64, 7, padding=3),
            nn.MaxPool1d(2, stride=2),
            nn.ReLU(),
            nn.Conv1d(64, 128, 3, padding=1),
            nn.MaxPool1d(2, stride=2),
            nn.ReLU(),  # the size is now 128 x input_size/4 x 1
        )

        self.fourier_conv_to_hidden = nn.Sequential(
            nn.Linear(128 * int(self.fourier_input_size / 4), self.hidden_size),
            nn.ReLU()
        )

        self.hidden_to_out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, gait, fourier):
        gait = gait.squeeze(0)
        gait = self.gait_to_hidden(gait)
        gait = gait.view(-1, num_flat_features(gait))
        
        
        fourier = fourier.squeeze(0)
        fourier = self.fourier_to_hidden(fourier)
        fourier = fourier.view(-1, num_flat_features(fourier))

        hidden = self.gait_conv_to_hidden(gait) + self.fourier_conv_to_hidden(fourier)

        out = self.hidden_to_out(hidden)
        return Funct.softmax(out)

In [4]:
from torch.utils.data import Dataset
import numpy as np
import pandas as pd


def get_other_data(dataset):
    """
    Generates a dictionary of data from a dataset in a specific way. Written to extract data from 
    the style of Pocket.csv (which is a dataset I found online).
    
    :param dataset: String that is the name/path of the dataset we want to extract
    :return: Dictionary of data, t, x, y, z
    """
    data = pd.read_csv(dataset)
    data = data.loc[data['Activity_Label']=="Walking"]
    t = np.linspace(0, len(data['Time_Stamp'])/50, len(data['Time_Stamp']))  # generate time data
    # 50 samples per second - start at zero and just count up.
    x = np.array(data['Ax'])
    y = np.array(data['Ay'])
    z = np.array(data['Az'])
    
    total_time = t[-1] - t[0]
    ends = 0.1 * total_time  # discard beginning and end of dataset because of irregularities

    t_start = t[0] + ends
    t_end = t[-1] - ends

    start_idx = np.where(t <= t_start)[0][-1]
    end_idx = np.where(t >= t_end)[0][0]

    t = t[start_idx:end_idx]
    x = x[start_idx:end_idx]
    y = -y[start_idx:end_idx]  # In the study where they took this data it also looks like they oriented
    z = -z[start_idx:end_idx]  # the data differently from [1] so this is needed.

    out_dict = {
        't': t,
        'x': x,
        'y': y,
        'z': z
    }
    return out_dict


def gen_fake_fourier(len, start, stop):
    """
    Generates a fake fourier component dataset. Written to make it look like my own data
    to try and fool the network.
    
    :param len: Length of the data to output
    :param start: Frequency start
    :param stop: Frequency stop
    :return: Array of data with 5 peaks that looks like my own accelerometer data.
    """
    x = np.linspace(start, stop, len)
    span = stop-start

    def lorentzian(x, gamma, center, A):
        return A / np.pi * (0.5 * gamma) / ((x - center)**2 + (0.5 * gamma)**2)

    out = lorentzian(x, 0.21, start + span/6 + np.random.randn() * 0.05, .2)
    out += lorentzian(x, 0.21, start + 2 * span / 6 + np.random.randn() * 0.05, 1)
    out += lorentzian(x, 0.21, start + 3 * span / 6 + np.random.randn() * 0.05, .8)
    out += lorentzian(x, 0.21, start + 4 * span / 6 + np.random.randn() * 0.05, .8)
    out += lorentzian(x, 0.21, start + 5 * span / 6 + np.random.randn() * 0.05, .2)

    return out


class GaitDataset(Dataset):
    """
    A subclass of torch.utils.data.Dataset that takes the cycles I've extracted 
    and formats them properly so that I can use the torch.utils.data.DataLoader to 
    feed data into the CNN.
    """
    def __init__(self, 
                 true_gait_samples, 
                 false_gait_samples, 
                 min_gait_len, 
                 true_fourier_sample, 
                 true_fourier_freqs,
                 false_fourier_sample,
                 false_fourier_freqs,
                 min_fourier_len):
        super(GaitDataset, self).__init__()

        self.true_gait_samples = true_gait_samples
        self.false_gait_samples = false_gait_samples
        self.true_fourier_sample = true_fourier_sample
        self.true_fourier_freqs = true_fourier_freqs
        self.false_fourier_sample = false_fourier_sample
        self.false_fourier_freqs = false_fourier_freqs
        self.min_gait_len = min_gait_len
        self.min_fourier_len = min_fourier_len

    def __getitem__(self, idx):
        p = np.random.randint(1, 100+1)
        if p > 70:  # 30% probability to return a sample of the owner (real)
            randindex = np.random.randint(0, len(self.true_gait_samples))

            gait = torch.from_numpy(
                self.true_gait_samples[randindex][:self.min_gait_len]  # make sure they're all the same length
            ).float().unsqueeze(0).unsqueeze(1)
            gait += torch.rand(*gait.size())*1e-4  # add some jitter

            fourier = torch.from_numpy(
                self.true_fourier_sample[:self.min_fourier_len]
            ).float().unsqueeze(0).unsqueeze(1)  # unsqueeze to give 3D input
            fourier += torch.rand(*fourier.size())*1e-3  # add some jitter

            label = torch.FloatTensor(np.array([1.0, 0.0]))  # sample from owner
            return gait, fourier, label
        else:  # 70% probability to return a sample not from the owner
            randindex = np.random.randint(0, len(self.false_gait_samples))

            fake_gait = torch.from_numpy(
                self.false_gait_samples[randindex][:self.min_gait_len]
            ).float().unsqueeze(0).unsqueeze(1)
            fake_gait += torch.rand(*fake_gait.size())*1e-4  # add some jitter

            fake_fourier = torch.from_numpy(
                self.false_fourier_sample[:self.min_fourier_len]  # make sure they're all the same length
            ).float().unsqueeze(0).unsqueeze(1)  # unsqueeze to give 3D input
            fake_fourier += torch.rand(*fake_fourier.size())*1e-3  # add some jitter

            label = torch.FloatTensor(np.array([0.0, 1.0]))  # sample from someone else
            return fake_gait, fake_fourier, label

    def __len__(self):
        return int(len(self.true_gait_samples) + len(self.false_gait_samples))

In [5]:
# Plot the data I took vs the data I downloaded for context

# Own (data that I took)
data_dict_own = generate_and_preprocess("data/longwalk.csv")
data_dict_other = get_other_data("data/Pocket.csv")

lf_own = apply_l_decomp(data_dict_own['z'], 3)
lf_own = apply_l_recon(lf_own, 3)

peaks_own = filter_true_peaks(lf_own)
avg_cycle_own, avg_index_own, dt_avg_own, cycles_own, time_own = extract_avg_cycle(peaks_own, 
                                                                                   data_dict_own['t'], 
                                                                                   lf_own)
freqs_own, fourier_own = extract_fourier_features(lf_own, dt_avg_own, max_f=6)

# Other (data that I downloaded)
lf_other = apply_l_decomp(data_dict_other['z'], 2)
lf_other = apply_l_recon(lf_other, 2)

peaks_other = filter_true_peaks(lf_other, 0.333)
avg_cycle_other, avg_index_other, dt_avg_other, cycles_other, time_other = extract_avg_cycle(peaks_other, 
                                                                                             data_dict_other['t'], 
                                                                                             lf_other)
freqs_other, fourier_other = extract_fourier_features(lf_other, dt_avg_other, max_f=6)

min_own = np.min([len(cycle) for cycle in cycles_own])
for i, cycle in enumerate(cycles_other, 0):
    if len(cycle) < min_own:  # there are a couple really short ones, get rid of those.
        cycles_other.pop(i)

N = len(cycles_other)
p = figure(height=500, width=500)
p2 = figure(height=500, width=500)
start = time_other[avg_index_other][0]
for i in range(N):
    if i == avg_index_other:
        continue
    else:
        length = np.min([len(time_other[avg_index_other]), len(cycles_other[i])])
        p.line(time_other[avg_index_other][:length]-start, cycles_other[i][:length], alpha=0.5)
p.line(time_other[avg_index_other]-start, avg_cycle_other, line_width=3, color='orange', alpha=0.9, legend="Average Gait Cycle")
p.xaxis.axis_label = "Time, Normalized to Average Cycle (s)"
p.yaxis.axis_label = "Z acceleration, Normalized (~ m/s^2)"
p.title.text = "Gait Cycles (Other Dataset)"
p.legend.location = 'bottom_left'


p2.line(freqs_other, fourier_other)
p2.xaxis.axis_label = "Frequency (Hz)"
p2.yaxis.axis_label = "Response (arb)"
p2.title.text = "Fourier Features of smoothed Z acceleration (Other Dataset)"

N = len(cycles_own)
p3 = figure(height=500, width=500)
p4 = figure(height=500, width=500)
start = time_own[avg_index_own][0]
for i in range(N):
    if i == avg_index_own:
        continue
    else:
        length = np.min([len(time_own[avg_index_own]), len(cycles_own[i])])
        p3.line(time_own[avg_index_own][:length]-start, cycles_own[i][:length], alpha=0.5)
p3.line(time_own[avg_index_own]-start, avg_cycle_own, line_width=3, color='orange', alpha=0.9, legend="Average Gait Cycle")
p3.xaxis.axis_label = "Time, Normalized to Average Cycle (s)"
p3.yaxis.axis_label = "Z acceleration, Normalized (~ m/s^2)"
p3.title.text = "Gait Cycles (Own Dataset)"
p3.legend.location = 'bottom_left'


p4.line(freqs_own, fourier_own)
p4.xaxis.axis_label = "Frequency (Hz)"
p4.yaxis.axis_label = "Response (arb)"
p4.title.text = "Fourier Features of smoothed Z acceleration (Own Dataset)"

s1 = gridplot([[p, p2], [p3, p4]])
show(s1, notebook_handle=True)

In [7]:
# Now train up our CNN
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

min_cycle_len = np.min(np.min([[len(cycle) for cycle in cycles_own], [len(cycle) for cycle in cycles_other]]))
fourier_len = len(fourier_own)

dataset = GaitDataset(cycles_own, 
                      cycles_other, 
                      min_cycle_len, 
                      fourier_own, 
                      freqs_own, 
                      fourier_other, 
                      freqs_other, 
                      fourier_len)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=4)

max_epochs = 5

gid = GaitID(min_cycle_len, fourier_len)

force_retrain = False

if force_retrain:
    import os
    os.remove("gait_id.dat")

try:
    gid.load_state_dict(torch.load("gait_id.dat"))
except FileNotFoundError or AttributeError or RuntimeError:
    criterion = nn.MSELoss()
    optimizer = optim.Adam(gid.parameters(), lr=1e-3, weight_decay=0.001)

    for epoch in range(max_epochs):

        running_loss = 0.0
        for i, data in enumerate(data_loader, 0):
            gait, four, label = data  # [1.0, 0.0] is own, [0.0, 1.0] is other
            gait = Variable(gait)
            four = Variable(four)
            label = Variable(label)

            gid.zero_grad()
            output = gid(gait, four)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]

            if i % 100 == 99:
                print('[epoch: %d, i: %5d] average loss: %.3f' % (epoch + 1, i + 1,
                                                                  running_loss / 100))
                running_loss = 0.0

    torch.save(gid.state_dict(), "gait_id.dat")
    print("Finished Training")

rand_sample_other = np.random.randint(0, len(cycles_other))
test_gait_other = to_Variable(cycles_other[rand_sample_other][:min_cycle_len])
test_fourier_other = to_Variable(fourier_other[:fourier_len])

rand_sample_own = np.random.randint(0, len(cycles_own))
test_gait_own = to_Variable(cycles_own[rand_sample_own][:min_cycle_len])
test_fourier_own = to_Variable(fourier_own[:fourier_len])

import timeit

print("Probability outputs on random data from dataset")
start = timeit.default_timer()
out_other = gid(test_gait_other, test_fourier_other)
out_own = gid(test_gait_own, test_fourier_own)
end = timeit.default_timer()
print("Inference for 2 samples took {:4.2f} ms".format((end - start) * 1e3))
print("(Data from other) {:4.2f}% own, {:4.2f}% other".format(out_other.data[0, 0]*100, out_other.data[0, 1]*100))
print("(Data from own) {:4.2f}% own, {:4.2f}% other".format(out_own.data[0, 0]*100, out_own.data[0, 1]*100))

Probability outputs on random data from dataset
Inference for 2 samples took 5.47 ms
(Data from other) 0.40% own, 99.60% other
(Data from own) 97.66% own, 2.34% other
